In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Pegando a data base

In [50]:
df_portugues = pd.read_csv('DB_port.csv', index_col=0)

In [51]:
n_linha = 3599  # 3600 passa a ser 1
print(df_portugues['text'][n_linha])  # idexação esta correta
print(df_portugues['reality'][n_linha])

autor de best-seller cita 5 expressões que indicam que "você não tem inteligência emocional".  a lista foi publicada no site linkedin por travis bradberry , autor do best-seller inteligência emocional 2.0.  inteligência emocional é um conceito presente na psicologia e criado pelo psicólogo norte-americano daniel goleman . uma pessoa com inteligência emocional é aquela que possui uma aptidão para lidar com as emoções. goleman subdivide a inteligência emocional em 5 habilidades: autoconhecimento emocional, controle emocional, automotivação, empatia e desenvolvimento de relacionamentos interpessoais. voltando ao livro inteligência emocional 2.0 ... o autor diz que vivemos atualmente num mundo em que as pessoas parecem não ouvir, não esperam sua vez de falar e cada vez mais voltam o foco para si própria (egocentrismo) ... estamos perdendo a capacidade de enxergar o outro. bradberry sugere que algumas expressões podem atrapalhar nossos relacionamentos, em especial os relacionamentos profiss

# Separando as listas de textos e labels

In [52]:
textos_totais = df_portugues['text']  # lista do pandas - ver se funciona - video usa lista normal
labels = df_portugues['reality']

# Definindo o recorte de treino e de teste

In [53]:
'''
tamanho_do_treino = 5800
tamanho_do_teste = len(labels) - tamanho_do_treino

# definindo o corte do treino e teste
dados_treino = textos_totais[700:tamanho_do_treino+700]
dados_teste = textos_totais[0:700]
labels_treino = labels[0:tamanho_do_treino]
labels_teste = labels[tamanho_do_treino:]
'''

'\ntamanho_do_treino = 5800\ntamanho_do_teste = len(labels) - tamanho_do_treino\n\n# definindo o corte do treino e teste\ndados_treino = textos_totais[700:tamanho_do_treino+700]\ndados_teste = textos_totais[0:700]\nlabels_treino = labels[0:tamanho_do_treino]\nlabels_teste = labels[tamanho_do_treino:]\n'

In [54]:
dados_treino, dados_teste, labels_treino, labels_teste = train_test_split(textos_totais, labels, test_size=0.25)

In [55]:
print(textos_totais[5620])
print(labels[5620])
print(dados_teste)
print(labels_teste)

febre amarela: por que são paulo virou área de risco para a oms e o que isso implica para o brasil . oms recomenda vacina para viajantes que visitarem todo o estado de são paulo; especialistas vêm ação como um alerta para expansão da vacinação, hoje com reforço em alguns municípios. .  a recomendação da organização mundial de saúde, que incluiu todo o estado de são paulo em área de risco de febre amarela , reforça que o brasil deverá se preparar para não só expandir a vacinação no estado de são paulo -- que hoje está em municípios específicos -- mas para uma maior expansão pelo brasil, dizem especialistas. a categorização como área de risco em todo o estado de são paulo foi feita porque há um risco real de transmissão na região, explica marcos boulos, coordenador de controle de doenças da secretaria de saúde do estado de são paulo. antes, com o vírus mais comum na região norte e nordeste, não havia transmissão registrada no estado -- cenário que mudou desde 2016, com o registro do víru

# Tokenizando as palavras mais importantes de treino

In [56]:
token = Tokenizer(num_words = 2000, oov_token='<OOV>')  # n de palavras a serem tokenizadas - token <oov> para não encontradas
token.fit_on_texts(dados_treino)  # tokeniza as mais importantes dos dados de TREINO
indice_palavras = token.word_index  # cria um indice das palavras tokenizadas (tem forma de dict)
print(indice_palavras)

 37879, 'cumpanhero': 37880, 'precavido': 37881, 'massiva': 37882, 'vegana': 37883, 'saborosos': 37884, 'imitações': 37885, 'meadow': 37886, 'fantásticos': 37887, 'bioquímicos': 37888, 'heme': 37889, 'músculo': 37890, 'mexidos': 37891, 'bioquímicas': 37892, 'fda': 37893, 'carboidratos': 37894, '“alimentos': 37895, 'infusão': 37896, 'desconexa': 37897, 'farofeiros': 37898, 'espelham': 37899, 'atravessado': 37900, 'zeneida': 37901, 'dosando': 37902, 'incompletos': 37903, 'impiedosos': 37904, 'preserve': 37905, 'viu”': 37906, 'ame': 37907, 'perseguiam': 37908, 'sustentaria': 37909, 'delegar': 37910, 'welles': 37911, '100mulheres': 37912, 'heezen': 37913, 'wanda': 37914, 'merced': 37915, 'moursli': 37916, 'brasa': 37917, 'föppel': 37918, 'dvbr': 37919, 'rowan': 37920, 'caçadores': 37921, 'compacta': 37922, 'mariah': 37923, 'bentes': 37924, 'hemoam': 37925, 'leuginase': 37926, 'lla': 37927, 'empresinha': 37928, 'condinome': 37929, 'elpídio': 37930, 'confiscando': 37931, 'jaguariúna': 37932,

# Sequenciando tokens de treino

In [68]:
sequencias_treino = token.texts_to_sequences(dados_treino)  # organizadas os tokens de treino em sequencia numéricas
print(sequencias_treino)

[[2, 3, 4, 5], [2, 3, 4, 6], [7, 8, 9, 10, 11, 2, 3, 12]]


# Adicionando padding no final das frases para que todas tenha o mesmo tamanho (max = 300 palavras)

In [58]:
# o que fazer quando a frase tem tamanhos diferentes? avançada: RaggedTensor / simples: padding
# comando abaixo faz com que todas as frases tenham o mesmo tamanho da maior entre elas
com_padding_treino = pad_sequences(sequencias_treino, padding='post',  # podemos truncar palavras axima do max length
                            maxlen=300, truncating='post')  # podemos especificar o tamnho maximo das palavras espaçadas
print(com_padding_treino)  # post faz com que o padding fique no final

[[   1    1   64 ...    0    0    0]
 [1777    1    1 ...    0    0    0]
 [   1    1    1 ...    0    0    0]
 ...
 [   4    1    2 ...    1    6 1266]
 [   1 1370  921 ...   14   16   12]
 [ 150    1    1 ... 1648    2  279]]
[[   1   64    5 ...   63 1012   14]
 [ 251    1    1 ...   11  732  346]
 [   3    1    7 ...  284    2  671]
 ...
 [   1    1    3 ...    0    0    0]
 [  15 1252    8 ...    0    0    0]
 [ 326    3  772 ...    0    0    0]]
[[   3 1192  156 ...    9  548    3]
 [   1    1    1 ...    1    7    1]
 [ 545   19    1 ...    1    2    1]
 ...
 [  63  365   44 ...    0    0    0]
 [ 358   40  929 ...    5    4    1]
 [  49    7 1917 ...   14    1    1]]
[[ 103    2   34 ...    3  899   13]
 [  87    1  241 ...  194   57  906]
 [ 586  996  989 ...    0    0    0]
 ...
 [   3   94   46 ... 1198   15 1382]
 [  18 1113    5 ...   28    1   20]
 [ 286  613   65 ...    0    0    0]]
[[ 617    8  407 ...    0    0    0]
 [   1   15  747 ...   12  685   13]
 [ 254    1   

In [59]:
print(com_padding_treino[0])  # ver a primeira sequencia numerica
print(com_padding_treino.shape)  # ver quantas sequencias temos e de que tamanho são

[  55   42    1   13    3  199    8  131   55    6  196  361   37    4
  480  573    8  127   84   85    1  208   55    1 1976    4 1800   25
   27    6   50    1    4  544    2   35  260    1   13   18 1726    5
    3  199    8  131   75    1    9   53   49    4   27  208   55  356
    1   10 1489   18 1120   32 1029    5    4    1  658   13  109  504
    1   23  410    8  127   84   85   54    1    7   27    3  489   16
    1    3    1    1  956    2   55    1    5    4  171   16    1   45
    5   22 1029    1    4   49   25   12  185    6   20   51    1   16
  648    1    3    1   32  693   10   14 1102    1    1    9   93    2
  199  548 1490 1423  213   33  579    8  131   79   61    1    9  271
  103   11 1353    2    1    2  243    1  835    6 1147    2   55   18
  250   54    4   33  579    8  807    1  119    2   12  262  608    2
   52  272   91    1    3   55   15  225  325    1   77    2  650   14
    1    1   50    2   93   13 1423  213    4  103   21    1    9   12
    1 

# Adicionando padding nas frases de teste para terem o mesmo formato das de treino

In [60]:
sequencias_teste = token.texts_to_sequences(dados_teste)  # organizadas os tokens de teste em sequencia numéricas
com_padding_teste = pad_sequences(sequencias_teste, padding='post',  # podemos truncar palavras axima do max length
                            maxlen=300, truncating='post')  # podemos especificar o tamnho maximo das palavras espaçadas
print(com_padding_teste)  # post faz com que o padding fique no final

[[ 919   89    1 ...    0    0    0]
 [ 316    6  494 ...  277   13   18]
 [   1    3    1 ...    0    0    0]
 ...
 [   1   34   19 ...    0    0    0]
 [ 328  610   41 ...   11  466   16]
 [  15 1620    2 ...    1   31  671]]


# Tranformando as lista em np arrays (necessário para funcionar)

In [61]:
com_padding_treino = np.array(com_padding_treino)
labels_treino = np.array(labels_treino)
com_padding_teste = np.array(com_padding_teste)
labels_teste = np.array(labels_teste)

# Criando a rede neural

In [62]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(2000, 16, input_length=300),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Algumas infos sobre o modelo

In [63]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 16)           32000     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 25        
Total params: 32,433
Trainable params: 32,433
Non-trainable params: 0
_________________________________________________________________


# Definindo o número de repetições e rodando o modelo

In [64]:
num_epochs = 10
history = model.fit(com_padding_treino, labels_treino, epochs=num_epochs,
                    validation_data=(com_padding_teste, labels_teste), verbose=2)

Epoch 1/10
169/169 - 3s - loss: 0.5627 - accuracy: 0.7844 - val_loss: 0.3735 - val_accuracy: 0.8872
Epoch 2/10
169/169 - 3s - loss: 0.2653 - accuracy: 0.9076 - val_loss: 0.2214 - val_accuracy: 0.9128
Epoch 3/10
169/169 - 3s - loss: 0.1875 - accuracy: 0.9265 - val_loss: 0.1933 - val_accuracy: 0.9350
Epoch 4/10
169/169 - 6s - loss: 0.1598 - accuracy: 0.9435 - val_loss: 0.1771 - val_accuracy: 0.9406
Epoch 5/10
169/169 - 2s - loss: 0.1382 - accuracy: 0.9531 - val_loss: 0.1678 - val_accuracy: 0.9433
Epoch 6/10
169/169 - 2s - loss: 0.1205 - accuracy: 0.9637 - val_loss: 0.1565 - val_accuracy: 0.9450
Epoch 7/10
169/169 - 3s - loss: 0.1058 - accuracy: 0.9702 - val_loss: 0.1517 - val_accuracy: 0.9450
Epoch 8/10
169/169 - 4s - loss: 0.0931 - accuracy: 0.9739 - val_loss: 0.1500 - val_accuracy: 0.9467
Epoch 9/10
169/169 - 7s - loss: 0.0816 - accuracy: 0.9778 - val_loss: 0.1470 - val_accuracy: 0.9478
Epoch 10/10
169/169 - 6s - loss: 0.0715 - accuracy: 0.9815 - val_loss: 0.1398 - val_accuracy: 0.9517